In [1]:
import pickle
import requests
import json
import pickle
import time
import random
from tqdm import tqdm
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
def get_replies(news_id, article_id, comment_url, parentno, count):
    reply_api_url = "https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=news&templateId=default_world&pool=cbox5&_cv=20230925155059&_callback=jQuery33108418242169677335_1696224307224&lang=ko&country=KR&objectId=news" + news_id + "%2C" + article_id + "&categoryId=&pageSize=20&indexSize=10&groupId=&listType=OBJECT&pageType=more&parentCommentNo=" + parentno + "&page=1&userType=&includeAllStatus=true&moreType=next&_=1696224307230"

    headers = {'User-agent': 'Mozilla/5.0', 'referer': comment_url}
    r = requests.get(reply_api_url, headers=headers)

    reply_raw_data = r.text
    reply_start_index = reply_raw_data.find('(') + 1
    reply_main_data = reply_raw_data[reply_start_index:-2]
    reply_main_data_dict = json.loads(reply_main_data)
    
    replies = {}
    if 'commentList' in reply_main_data_dict['result']:
        reply_count = 0
        for reply_dict in reply_main_data_dict['result']['commentList']:
            key = f"{count}_{reply_count}"  
            replies[key] = {}
            replies[key]['parentCommentNo'] = parentno  
            replies[key]['userId'] = reply_dict['userName']
            replies[key]['Time'] = reply_dict['modTime']
            replies[key]['content'] = reply_dict['contents']
            replies[key]['sympathyCount'] = reply_dict['sympathyCount']
            replies[key]['antipathyCount'] = reply_dict['antipathyCount']
            reply_count += 1

    return replies

In [3]:
def get_comments(url):
    news_id = url.split("/")[4]
    article_id = url.split('/')[5].split('?')[0]
    
    comment_url = url
    total_comments = {}
    total_replies = {}

    page = 1
    moreParam_prev = ""
    moreParam_next = ""
    while True:
        sleep_time = random.random()*1.5
        time.sleep(sleep_time)
        
        # 공감순은 sort=FAVORITE로, 최신준 정렬은 sort=new로 변경할 것
        comment_api_url = f"https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=news&templateId=view_society_m1&pool=cbox5&_callback=jQuery33109493201888925509_1696217590564&lang=ko&country=KR&objectId=news{news_id}%2C{article_id}&pageSize=20&indexSize=10&groupId=&listType=OBJECT&pageType=more&page={page}&refresh=false&sort=FAVORITE&moreParam.prev={moreParam_prev}&moreParam.next={moreParam_next}"
        
        headers = {'User-agent': 'Mozilla/5.0', 'referer': comment_url}
        r=requests.get(comment_api_url, headers=headers)
        
        raw_data = r.text
        start_index = raw_data.find('(') + 1
        main_data = raw_data[start_index:-2]
        main_data_dict = json.loads(main_data)
        
        # 페이지 네비게이션 정보 업데이트
        page = main_data_dict['result']['pageModel']['nextPage']
        moreParam_prev = main_data_dict['result']['morePage']['prev']
        moreParam_next = main_data_dict['result']['morePage']['next']
        
        # 댓글 정보 추출
        count = len(total_comments)
        for comment_dict in main_data_dict['result']['commentList']:
            total_comments[str(count)] = {}
            total_comments[str(count)]['userId'] = comment_dict['userName']
            total_comments[str(count)]['Time'] = comment_dict['modTime']
            total_comments[str(count)]['content'] = comment_dict['contents']
            total_comments[str(count)]['sympathyCount'] = comment_dict['sympathyCount']
            total_comments[str(count)]['antipathyCount'] = comment_dict['antipathyCount']
            total_comments[str(count)]['replyCount'] = comment_dict['replyCount']
            total_comments[str(count)]['CommentNo'] = comment_dict['parentCommentNo']
                      
            # get_replies 함수 호출하기:
            if comment_dict['replyCount'] > 0:
                parentno = comment_dict['parentCommentNo']
                replies_for_comment = get_replies(news_id, article_id, comment_url, parentno, count)
                total_replies.update(replies_for_comment)
        
            count += 1
        
        # 다음 페이지가 없으면 종료
        if page == 0 or page == main_data_dict['result']['pageModel']['totalPages']:
            break
        
    return total_comments, total_replies

In [4]:
from datetime import datetime

# 문자열을 datetime 객체로 변환
def convert_time_format(time_str):
    dt = datetime.strptime(time_str, '%Y-%m-%dT%H:%M:%S%z')
    return dt.strftime('%Y-%m-%d %H:%M:%S')

## 기사 댓글 수집하기

In [5]:
data = pd.read_csv('../data_crawling/news_final/hangyoreh_final.csv', index_col=0)
# 중복기사 삭제
data = data.drop_duplicates(subset='link', keep='first')

# 3월 데이터만 가져오기
hangyoreh3 = data[data['time'].str.startswith(('2023-03'))]
hangyoreh3['index'] = range(len(hangyoreh3))

# 'index' 칸을 DataFrame의 인덱스로 지정
hangyoreh3 = hangyoreh3.set_index('index')
hangyoreh3.index = hangyoreh3.index+1
hangyoreh3

<ipython-input-5-d1ca18ff5094>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hangyoreh7['index'] = range(len(hangyoreh7))


,title,link,time,contents,category
index,,,,,
1,윤 대통령은 “반국가 세력” 검찰총장이었나,https://n.news.naver.com/article/028/000264631...,2023-07-02 07:33:01,성한용 선임기자의 정치 막전막후 487말·생각·인사 모두 극우로 치닫는 윤문 전 대...,정치
2,"이낙연, 이재명 작심 비판 “민주당 혁신 핵심은 도덕성·민주주의”",https://n.news.naver.com/article/028/000264635...,2023-07-02 15:55:03,이낙연 전 더불어민주당 대표가 2일 오전 광주 북구 국립 5·18 민주묘지를 참배하...,정치
3,민주 “쿠데타”-국힘 “마약 도취”…정치권 ‘막말 인플레이션’,https://n.news.naver.com/article/028/000264637...,2023-07-02 18:21:05,윤 대통령 “반국가 세력” 불 당기자민주 “사실상 쿠데타로 집권” 맞불김기현 “마약...,정치
4,김기현 “문재인 정부 ‘KBS·MBC 장악 시도’ 수사해야”,https://n.news.naver.com/article/028/000264640...,2023-07-02 21:51:01,김기현 국민의힘 대표가 지난 30일 국회에서 열린 청년정책네트워크 3호 정책 ‘개인...,정치
5,"윤 대통령 “통일부, ‘북한지원부’ 아니다…이제는 달라져야”",https://n.news.naver.com/article/028/000264634...,2023-07-02 14:51:07,윤석열 대통령이 지난 28일 청와대 영빈관에서 열린 2023 국가재정전략회의에서 발...,정치
...,...,...,...,...,...
174,"장모 구속·양평 땅 의혹 커지는데…대통령실, 특별감찰관 ‘모르쇠’",https://n.news.naver.com/article/028/000265044...,2023-07-31 18:17:05,윤석열 대통령과 김건희 여사가 유엔군 참전의 날 및 정전협정 70주년을 맞은 지난 ...,정치
175,"윤 대통령, 다음달 2∼8일 여름휴가…‘저도의 추억’ 쌓는다",https://n.news.naver.com/article/028/000265040...,2023-07-31 15:59:05,윤석열 대통령이 27일 부산 중구 자갈치시장에서 장어를 직접 손으로 잡아 보며 즐거...,정치
176,‘윤 입맛대로’ 통일부 개악…어느 정부서도 볼 수 없던 퇴행,https://n.news.naver.com/article/028/000265032...,2023-07-30 20:03:09,김영호 신임 통일부 장관이 28일 정부서울청사에서 열린 취임식에서 취임사를 하고 있...,정치


In [6]:
hangyoreh3['link']

index
1      https://n.news.naver.com/article/028/000264631...
2      https://n.news.naver.com/article/028/000264635...
3      https://n.news.naver.com/article/028/000264637...
4      https://n.news.naver.com/article/028/000264640...
5      https://n.news.naver.com/article/028/000264634...
                             ...                        
174    https://n.news.naver.com/article/028/000265044...
175    https://n.news.naver.com/article/028/000265040...
176    https://n.news.naver.com/article/028/000265032...
177    https://n.news.naver.com/article/028/000265042...
178    https://n.news.naver.com/article/028/000265035...
Name: link, Length: 178, dtype: object

In [7]:
for i, link in tqdm(enumerate(hangyoreh3['link'], 1), total=len(hangyoreh3['link'])): # 1부터 시작하여 i값 증가
    try:
        total_comments, total_replies = get_comments(link)
        
        # 데이터 프레임으로 변환
        df_comments = pd.DataFrame.from_dict(total_comments, orient='index')
        df_replies = pd.DataFrame.from_dict(total_replies, orient='index')
        
        # 'Time' 컬럼의 시간 형식 변경
        if not df_comments.empty:
            df_comments['Time'] = df_comments['Time'].apply(convert_time_format)

        if not df_replies.empty:
            df_replies['Time'] = df_replies['Time'].apply(convert_time_format)

        # 삭제된 댓글(빈 값이 들어가 있는 행) 삭제
#         df_comment.dropna(inplace=True)
#         df_replies.dropna(inplace=True)
        
        # CSV 파일로 저장
        df_comments.to_csv(f'../data_crawling/comments/FAVORITE/hangyoreh3_comments/comments_hangyoreh_{i}.csv', index=False)
        df_replies.to_csv(f'../data_crawling/comments/FAVORITE/hangyoreh3_replies/replies_hangyoreh_{i}.csv', index=False)
        
        print(f"Saved comments and replies for link {i}")
        
        # 잠시 대기
        time.sleep(random.random() * 1.5)
        
    except Exception as e:
        print(f"Failed to save comments and replies for link {i} due to {e}")

  0%|          | 0/178 [00:00<?, ?it/s]

Saved comments and replies for link 1


  1%|          | 1/178 [01:20<3:56:49, 80.28s/it]

Saved comments and replies for link 2


  2%|▏         | 3/178 [02:47<2:30:13, 51.51s/it]

...

 99%|█████████▉| 176/178 [1:31:15<00:48, 24.10s/it]

Saved comments and replies for link 177


 99%|█████████▉| 177/178 [1:31:26<00:20, 20.20s/it]

Saved comments and replies for link 178


100%|██████████| 178/178 [1:31:33<00:00, 30.86s/it]
